In [1]:
import numpy as np

def sin(x, T=100):
    return np.sin(2.0 * np.pi * x/T)

def toy_problem(T=100, ampl=0.05):
    x = np.arange(0, 2*T+1)
    noise = ampl * np.random.uniform(low=-1.0, high=1.0, size=len(x))
    return sin(x) + noise

In [2]:
# 예제 데이터셋 생성
T = 100
f = toy_problem(T)

In [3]:
# 시간설정: 25
length_of_sequences = 2 * T # 시계열 전체의 길이
maxlen = 25 # 하나의 시계열 데이터의 길이

data = []
target = []

for i in range(0, length_of_sequences - maxlen + 1):
    data.append(f[i:i+maxlen])
    target.append(f[i+maxlen])

In [4]:
X = np.zeros((len(data), maxlen, 1), dtype=float)
Y = np.zeros((len(data), 1), dtype=float)

for i, seq in enumerate(data):
    for t, value in enumerate(seq):
        X[i, t, 0] = value
    Y[i, 0] = target[i]

In [5]:
N_train = int(len(data) * 0.9)
N_validation = len(data) - N_train

from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, Y, test_size=N_validation)

### 이제 텐서플로로 구현해보자

In [6]:
import tensorflow as tf
import tensorflow

def inference(x, n_batch, maxlen=None, n_hidden=None, n_out=None):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)
    def bias_variable(shape):
        initial = tf.zeros(shape, dtype=tf.float32)
        return tf.Variable(initial)
    
    cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(n_hidden)
    initial_state = cell.zero_state(n_batch, tf.float32)
    
    state= initial_state
    outputs = []
    with tf.variable_scope('RNN'):
        for t in range(maxlen):
            if t > 0:
                tf.get_variable_scope().reuse_variables()
            (cell_output, state) = cell(x[:, t, :], state)
            outputs.append(cell_output)
    output = outputs[-1]
    
    V = weight_variable([n_hidden, n_out])
    c = bias_variable([n_out])
    y = tf.matmul(output, V)
    
    return y

In [7]:
def loss(y, t):
    mse = tf.reduce_mean(tf.square(y-t))
    return mse

In [8]:
def training(loss):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999)
    train_step = optimizer.minimize(loss)
    return train_step

In [9]:
n_in = len(X[0][0]) # 1
n_hidden = 20
n_out = len(Y[0])  # 1

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
x = tf.placeholder(tf.float32, shape=[None, maxlen, n_in])
t = tf.placeholder(tf.float32, shape=[None, n_out])
n_batch = len(X) // 10

y = inference(x, n_batch, maxlen=maxlen, n_hidden=n_hidden, n_out=n_out)
loss = loss(y, t)
train_step = training(loss)

Instructions for updating:
non-resource variables are not supported in the long term


ModuleNotFoundError: No module named 'keras.api._v1.keras.__internal__.legacy'

In [36]:
# 버전이 달라져서 자꾸 에러 생기는 듯... 다른 책으로 다시 찾아보기ㅜ

### Keras로 구현하기

In [13]:
from keras.layers import SimpleRNN

ImportError: cannot import name 'SimpleRNN' from 'keras.layers' (unknown location)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.layers.core import Dropout
from keras.optimizers import Adam

AssertionError: Duplicate registrations for type 'optimizer'

In [51]:
model = Sequential()
model.add(SimpleRNN(n_hidden, input_shape=(maxlen, n_out)))
model.add(Dense(n_out))
model.add(Activation('linear'))

In [54]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mean_squared_error', optimizer=optimizer)

C:\Users\Main\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [58]:
epochs = 100
batch_size = 10

with tf.device("/device:GPU:0"):
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validation, y_validation))

Train on 158 samples, validate on 18 samples
Epoch 1/100
158/158 [==============================] - 0s 2ms/sample - loss: 0.0011 - val_loss: 0.0017
Epoch 2/100
158/158 [==============================] - 0s 3ms/sample - loss: 8.0788e-04 - val_loss: 0.0012
Epoch 3/100
158/158 [==============================] - 0s 3ms/sample - loss: 7.3000e-04 - val_loss: 0.0010
Epoch 4/100
158/158 [==============================] - 0s 2ms/sample - loss: 8.2705e-04 - val_loss: 7.4514e-04
Epoch 5/100
158/158 [==============================] - 0s 2ms/sample - loss: 9.1026e-04 - val_loss: 0.0018
Epoch 6/100
158/158 [==============================] - 0s 3ms/sample - loss: 7.8679e-04 - val_loss: 0.0013
Epoch 7/100
158/158 [==============================] - 0s 3ms/sample - loss: 8.2158e-04 - val_loss: 0.0018
Epoch 8/100
158/158 [==============================] - 0s 2ms/sample - loss: 0.0010 - val_loss: 0.0017
Epoch 9/100
158/158 [==============================] - 0s 2ms/sample - loss: 8.0685e-04 - val_loss: 0.0